In [1]:
!python --version

Python 3.10.13


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os, time, datetime
import json, pickle
from pprint import pprint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import tree based models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

# training/testing utils
from sklearn.utils import resample
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight

# Symbolic regression
from gplearn.genetic import SymbolicClassifier

## Load and prepare data

In [4]:
data = pd.read_csv('../data/data_diet_filtered.csv', index_col=0)

data.disease, enc_values = pd.factorize(data.disease) # to get back original labels, do "enc_values[data.disease]"

drop_col = ['index', 'disease', 'subject_id','gender', 'country','age_category', 'diet']
X = data.drop(labels=drop_col, axis=1)
y = data.disease.values

class_map = {v: k for v, k in enumerate(list(enc_values.values))}  # coded label to names
class_counts = {k: len(y[y==v]) for v, k in enumerate(list(enc_values.values))}
assert sum([class_counts[k] for k in class_counts]) == len(X), 'total #samples not matching when summing for each class'

print(data.shape, X.shape, y.shape, class_map, class_counts)

(14560, 756) (14560, 749) (14560,) {0: 'healthy', 1: 'IBD', 2: 'CRC', 3: 'adenoma', 4: 'T2D'} {'healthy': 10761, 'IBD': 1736, 'CRC': 701, 'adenoma': 209, 'T2D': 1153}


/var/folders/qp/4w02sqhj6_d43815lhzjb7900000gn/T/ipykernel_73514/129412594.py:1: DtypeWarning: Columns (756) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/data_diet_filtered.csv', index_col=0)


In [5]:
# only take normalized rows!
normalized_idx = (X.sum(1) > 99)
X = X[normalized_idx]
y = y[X.index]

print(f"X.shape: {X.shape}, y.shape: {y.shape}")

X.shape: (14156, 749), y.shape: (14156,)


In [6]:
from srmb.utils import calculate_metrics

## Healthy vs. CRC classification
For the subsequent analysis, we will only choose the healthy and CRC patients from the 
dataset.

In [7]:
from srmb.fitness_functions import customacc
from srmb.special_functions import (
    presence, absence, add3, add10, ifelse, ifelseless,
    presence2, absence2,
    presence3, absence3
)

Error exception: Only one class present in y_true. ROC AUC score is not defined in that case.


In [8]:
k = 2
idxs = (y == 0) | (y == k)  # get healthy and that class' data

X1, y1 = X.iloc[idxs], y[idxs]
y1[y1 == k] = 1  # relabel 1 --> CRC, 0 --> healthy
print(f'doing for class = {class_map[k]}, {X1.shape=}, {y1.shape=}, #{class_map[k]} samples = {y1.sum()}')

doing for class = CRC, X1.shape=(11137, 749), y1.shape=(11137,), #CRC samples = 664


# 1. Experiments with undersampling the healthy class

In [9]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

In [10]:
def store_result(key, acc, f1, auroc):
    ACCURACIES[key].append(acc)
    F1SCORES[key].append(f1)
    AUCROCS[key].append(auroc)
    
def convert_to_arrays(prec=4):
    for key in MODEL_NAMES:
        ACCURACIES[key] = np.around(np.asarray(ACCURACIES[key]), prec)
        F1SCORES[key] = np.around(np.asarray(F1SCORES[key]), prec)
        AUCROCS[key] = np.around(np.asarray(AUCROCS[key]), prec)

In [11]:
# seeds for 20 trials
RANDOM_SEEDS_FOR_UNDERSAMPLING = [42, 2024, 1234, 2405, 11, 9345,  858, 8590, 4754, 1959,
                                  707, 10524, 83946, 63297, 78035, 22664, 49283, 35253, 82273, 90378]
MODEL_NAMES = ['LR', 'DT', 'RF', 'XG', 'SR', 'SRf']
ACCURACIES = {k: [] for k in MODEL_NAMES}
F1SCORES = {k: [] for k in MODEL_NAMES}
AUCROCS = {k: [] for k in MODEL_NAMES}

SRmodels = []
SRfmodels = []

In [12]:
USE_BALANCED_SUBSAMPLE=True # perform undersampling of healthy classes

for random_state in RANDOM_SEEDS_FOR_UNDERSAMPLING:
    rus = RandomUnderSampler(sampling_strategy=0.85, # this is another hyperparameter
                             random_state=random_state)
    X1b, y1b = rus.fit_resample(X1, y1)
    print(f'seed={random_state}')
    # print(sorted(Counter(y1b).items()))
    
    if not USE_BALANCED_SUBSAMPLE:
        X_train, X_test, y_train, y_test = train_test_split(X1, y1,
                                                        test_size=0.25,
                                                        # train_size=0.5, # if slow use this
                                                        random_state=random_state, stratify=y1)
        class_weight = compute_class_weight(class_weight='balanced', classes=np.unique(y1), y=y1)
    else:
        # print('using a balanced subsample of the data ...')
        X_train, X_test, y_train, y_test = train_test_split(X1b, y1b,
                                                            test_size=0.25,
                                                            # train_size=0.5, # if slow use this
                                                            random_state=42, stratify=y1b)
        class_weight = compute_class_weight(class_weight='balanced', classes=np.unique(y1b), y=y1b)
        
    
    sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)
    
    # logistic regression classifier
    model_lr = LogisticRegression(max_iter=500, class_weight=dict(enumerate(class_weight)))
    model_lr.fit(X_train, y_train)
    store_result('LR', *calculate_metrics(model_lr, X_train, y_train, X_test, y_test))
    
    # Decision tree classifier
    model_dt = DecisionTreeClassifier(max_depth=5, class_weight=dict(enumerate(class_weight)))
    model_dt.fit(X_train, y_train)
    store_result('DT', *calculate_metrics(model_dt, X_train, y_train, X_test, y_test))
    
    
    # Random forest classifier
    model_rf = RandomForestClassifier(n_estimators=50, class_weight=dict(enumerate(class_weight)))
    model_rf.fit(X_train, y_train)
    store_result('RF', *calculate_metrics(model_rf, X_train, y_train, X_test, y_test))
    
    
    # Create an XGBoost classifier for multiclass classification
    model_xg = XGBClassifier(n_estimators=50, max_depth=5, learning_rate=0.1, objective='binary:logistic')
    model_xg.fit(X_train, y_train, sample_weight=sample_weights)
    store_result('XG', *calculate_metrics(model_xg, X_train, y_train, X_test, y_test))
    
    
    # Do it for vanilla symbolic regression
    function_set = ['add', 'sub', 'mul', 'div', 'neg', 'max', 'min', 'sqrt', 'log']

    est = SymbolicClassifier(population_size=6000,
                             generations=20,
                             tournament_size=25,

                             init_depth=(2, 6),
                             const_range=(0., 100.),
                             # init_method="full",
                             parsimony_coefficient=0.001,
                             function_set=function_set,

                             stopping_criteria=1.0, metric=customacc, #use custom acc as fitness
                             
                             feature_names=X1.columns.to_list(),
                             # verbose=True,
                             random_state=42)

    t0 = time.time()
    est.fit(X_train, y_train)
    print('Time to fit symbolic classifier:', time.time() - t0, 'seconds')
    store_result('SR', *calculate_metrics(est, X_train, y_train, X_test, y_test))
    SRmodels.append(est)
    
    
    # SR with special functions
    special_functions = [presence, absence, presence2, absence2, ifelse]#, add3, add10]
    function_set = ['add', 'sub', 'mul', 'div', 'neg', 'max', 'min'] + special_functions

    est = SymbolicClassifier(population_size=6000,
                             generations=20,
                             tournament_size=25,

                             init_depth=(2, 6),
                             const_range=(0., 100.),
                             # init_method="full",
                             parsimony_coefficient=0.001,
                             function_set=function_set,

                             stopping_criteria=1.0, metric=customacc, #use custom acc as fitness
                             
                             feature_names=X1.columns.to_list(),
                             # verbose=True,
                             random_state=42)

    t0 = time.time()
    est.fit(X_train, y_train)
    print('Time to fit symbolic classifier:', time.time() - t0, 'seconds')
    store_result('SRf', *calculate_metrics(est, X_train, y_train, X_test, y_test))
    SRfmodels.append(est)

seed=42


/Users/swagatam/miniconda3/envs/hiwi/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression
Test accuracy: 0.7265 Test AUROC: 0.7981 Test F1 score: 0.7009
DecisionTreeClassifier
Test accuracy: 0.7514 Test AUROC: 0.8186 Test F1 score: 0.7078
RandomForestClassifier
Test accuracy: 0.8011 Test AUROC: 0.9015 Test F1 score: 0.7647
XGBClassifier
Test accuracy: 0.8260 Test AUROC: 0.9131 Test F1 score: 0.8013
Time to fit symbolic classifier: 42.2771680355072 seconds
SymbolicClassifier
Test accuracy: 0.7431 Test AUROC: 0.7488 Test F1 score: 0.6847
Time to fit symbolic classifier: 41.15023899078369 seconds
SymbolicClassifier
Test accuracy: 0.7514 Test AUROC: 0.7579 Test F1 score: 0.6939
seed=2024


/Users/swagatam/miniconda3/envs/hiwi/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression
Test accuracy: 0.7210 Test AUROC: 0.7726 Test F1 score: 0.6967
DecisionTreeClassifier
Test accuracy: 0.7182 Test AUROC: 0.7402 Test F1 score: 0.6982
RandomForestClassifier
Test accuracy: 0.8122 Test AUROC: 0.8961 Test F1 score: 0.7792
XGBClassifier
Test accuracy: 0.8177 Test AUROC: 0.9159 Test F1 score: 0.7911
Time to fit symbolic classifier: 41.51435089111328 seconds
SymbolicClassifier
Test accuracy: 0.7569 Test AUROC: 0.7640 Test F1 score: 0.6966
Time to fit symbolic classifier: 41.539920806884766 seconds
SymbolicClassifier
Test accuracy: 0.7541 Test AUROC: 0.7641 Test F1 score: 0.6962
seed=1234


...

LogisticRegression
Test accuracy: 0.7569 Test AUROC: 0.8184 Test F1 score: 0.7317
DecisionTreeClassifier
Test accuracy: 0.7514 Test AUROC: 0.8107 Test F1 score: 0.7059
RandomForestClassifier
Test accuracy: 0.8039 Test AUROC: 0.8988 Test F1 score: 0.7717
XGBClassifier
Test accuracy: 0.8094 Test AUROC: 0.9091 Test F1 score: 0.7903
Time to fit symbolic classifier: 44.62361788749695 seconds
SymbolicClassifier
Test accuracy: 0.7210 Test AUROC: 0.7384 Test F1 score: 0.6731
Time to fit symbolic classifier: 44.03053903579712 seconds
SymbolicClassifier
Test accuracy: 0.7403 Test AUROC: 0.7603 Test F1 score: 0.6928
seed=82273


/Users/swagatam/miniconda3/envs/hiwi/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression
Test accuracy: 0.7293 Test AUROC: 0.7904 Test F1 score: 0.7048
DecisionTreeClassifier
Test accuracy: 0.7403 Test AUROC: 0.7878 Test F1 score: 0.6908
RandomForestClassifier
Test accuracy: 0.8039 Test AUROC: 0.8927 Test F1 score: 0.7609
XGBClassifier
Test accuracy: 0.8425 Test AUROC: 0.9112 Test F1 score: 0.8190
Time to fit symbolic classifier: 45.13826107978821 seconds
SymbolicClassifier
Test accuracy: 0.7486 Test AUROC: 0.7551 Test F1 score: 0.6936
Time to fit symbolic classifier: 44.299291133880615 seconds
SymbolicClassifier
Test accuracy: 0.7486 Test AUROC: 0.7536 Test F1 score: 0.6915
seed=90378


/Users/swagatam/miniconda3/envs/hiwi/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression
Test accuracy: 0.7541 Test AUROC: 0.8200 Test F1 score: 0.7343
DecisionTreeClassifier
Test accuracy: 0.7541 Test AUROC: 0.8313 Test F1 score: 0.6877
RandomForestClassifier
Test accuracy: 0.8011 Test AUROC: 0.9094 Test F1 score: 0.7647
XGBClassifier
Test accuracy: 0.8287 Test AUROC: 0.9243 Test F1 score: 0.8025
Time to fit symbolic classifier: 44.41036295890808 seconds
SymbolicClassifier
Test accuracy: 0.7597 Test AUROC: 0.7663 Test F1 score: 0.7031
Time to fit symbolic classifier: 46.336766958236694 seconds
SymbolicClassifier
Test accuracy: 0.7597 Test AUROC: 0.7671 Test F1 score: 0.7031


## Process results

In [13]:
LENGTHS = {'SR': [], 'SRf': []}
for est in SRmodels:
    LENGTHS['SR'].append(est._program.length_)
for est in SRfmodels:
    LENGTHS['SRf'].append(est._program.length_)
    
LENGTHS['SR'] = np.asarray(LENGTHS['SR'])
LENGTHS['SRf'] = np.asarray(LENGTHS['SRf'])

convert_to_arrays()

In [14]:
plusminus = pm = u"\u00B1"
for model_name in MODEL_NAMES:
    print(f"{model_name} acc: {ACCURACIES[model_name].mean():.2f} {pm} {ACCURACIES[model_name].std():.4f}")
print('--------')
for model_name in MODEL_NAMES:
    print(f"{model_name} F1: {F1SCORES[model_name].mean():.2f} {pm} {F1SCORES[model_name].std():.4f}")

print('--------')

LR acc: 0.75 ± 0.0232
DT acc: 0.74 ± 0.0238
RF acc: 0.82 ± 0.0152
XG acc: 0.83 ± 0.0146
SR acc: 0.75 ± 0.0107
SRf acc: 0.75 ± 0.0145
--------
LR F1: 0.72 ± 0.0230
DT F1: 0.70 ± 0.0309
RF F1: 0.78 ± 0.0187
XG F1: 0.80 ± 0.0148
SR F1: 0.70 ± 0.0110
SRf F1: 0.69 ± 0.0166
--------


## Compare average length of SR models with and without special functions

In [35]:
print(f"SR mean length: {LENGTHS['SR'].mean()}")
print(f"SRf mean length: {LENGTHS['SRf'].mean()}")

SR mean length: 19.55
SRf mean length: 14.55


## Save symbolic regression models for later use

In [15]:
from srmb.utils import save_sr_models, load_sr_models

In [16]:
save_sr_models(SRmodels, key='SR', save_dir='../results_srmb/sr_vanilla_models/')
save_sr_models(SRfmodels, key='SRf', save_dir='../results_srmb/sr_special_models/')

## Load the models

In [27]:
SRMODELS = load_sr_models('SR', save_dir='../results_srmb/sr_vanilla_models/')
SRFMODELS = load_sr_models('SRf', save_dir='../results_srmb/sr_special_models/')

In [19]:
# SRMODELS

# Ablation: what if we use the entire imbalanced dataset?

In [20]:
# seeds for 20 trials
RANDOM_SEEDS_FOR_UNDERSAMPLING = [42, 2024, 1234, 2405, 11, 9345,  858, 8590, 4754, 1959,
                                  707, 10524, 83946, 63297, 78035, 22664, 49283, 35253, 82273, 90378]
MODEL_NAMES = ['LR', 'DT', 'RF', 'XG', 'SR', 'SRf']
ACCURACIES = {k: [] for k in MODEL_NAMES}
F1SCORES = {k: [] for k in MODEL_NAMES}
AUCROCS = {k: [] for k in MODEL_NAMES}

SRmodels = []
SRfmodels = []

In [21]:
USE_BALANCED_SUBSAMPLE=False # perform undersampling of healthy classes

for random_state in RANDOM_SEEDS_FOR_UNDERSAMPLING:
    rus = RandomUnderSampler(sampling_strategy=0.85, # this is another hyperparameter
                             random_state=random_state)
    X1b, y1b = rus.fit_resample(X1, y1)
    print(f'seed={random_state}')
    # print(sorted(Counter(y1b).items()))
    
    if not USE_BALANCED_SUBSAMPLE:
        X_train, X_test, y_train, y_test = train_test_split(X1, y1,
                                                        test_size=0.25,
                                                        # train_size=0.5, # if slow use this
                                                        random_state=random_state, stratify=y1)
        class_weight = compute_class_weight(class_weight='balanced', classes=np.unique(y1), y=y1)
    else:
        # print('using a balanced subsample of the data ...')
        X_train, X_test, y_train, y_test = train_test_split(X1b, y1b,
                                                            test_size=0.25,
                                                            # train_size=0.5, # if slow use this
                                                            random_state=42, stratify=y1b)
        class_weight = compute_class_weight(class_weight='balanced', classes=np.unique(y1b), y=y1b)
        
    
    sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)
    
    # logistic regression classifier
    model_lr = LogisticRegression(max_iter=500, class_weight=dict(enumerate(class_weight)))
    model_lr.fit(X_train, y_train)
    store_result('LR', *calculate_metrics(model_lr, X_train, y_train, X_test, y_test))
    
    # Decision tree classifier
    model_dt = DecisionTreeClassifier(max_depth=5, class_weight=dict(enumerate(class_weight)))
    model_dt.fit(X_train, y_train)
    store_result('DT', *calculate_metrics(model_dt, X_train, y_train, X_test, y_test))
    
    
    # Random forest classifier
    model_rf = RandomForestClassifier(n_estimators=50, class_weight=dict(enumerate(class_weight)))
    model_rf.fit(X_train, y_train)
    store_result('RF', *calculate_metrics(model_rf, X_train, y_train, X_test, y_test))
    
    
    # Create an XGBoost classifier for multiclass classification
    model_xg = XGBClassifier(n_estimators=50, max_depth=5, learning_rate=0.1, objective='binary:logistic')
    model_xg.fit(X_train, y_train, sample_weight=sample_weights)
    store_result('XG', *calculate_metrics(model_xg, X_train, y_train, X_test, y_test))
    
    
    # Do it for vanilla symbolic regression
    function_set = ['add', 'sub', 'mul', 'div', 'neg', 'max', 'min', 'sqrt', 'log']

    est = SymbolicClassifier(population_size=6000,
                             generations=20,
                             tournament_size=25,

                             init_depth=(2, 6),
                             const_range=(0., 100.),
                             # init_method="full",
                             parsimony_coefficient=0.001,
                             function_set=function_set,

                             stopping_criteria=1.0, metric=customacc, #use custom acc as fitness
                             
                             feature_names=X1.columns.to_list(),
                             # verbose=True,
                             random_state=42)

    t0 = time.time()
    est.fit(X_train, y_train)
    print('Time to fit symbolic classifier:', time.time() - t0, 'seconds')
    store_result('SR', *calculate_metrics(est, X_train, y_train, X_test, y_test))
    SRmodels.append(est)
    
    
    # SR with special functions
    special_functions = [presence, absence, presence2, absence2, ifelse]#, add3, add10]
    function_set = ['add', 'sub', 'mul', 'div', 'neg', 'max', 'min'] + special_functions

    est = SymbolicClassifier(population_size=6000,
                             generations=20,
                             tournament_size=25,

                             init_depth=(2, 6),
                             const_range=(0., 100.),
                             # init_method="full",
                             parsimony_coefficient=0.001,
                             function_set=function_set,

                             stopping_criteria=1.0, metric=customacc, #use custom acc as fitness
                             
                             feature_names=X1.columns.to_list(),
                             # verbose=True,
                             random_state=42)

    t0 = time.time()
    est.fit(X_train, y_train)
    print('Time to fit symbolic classifier:', time.time() - t0, 'seconds')
    store_result('SRf', *calculate_metrics(est, X_train, y_train, X_test, y_test))
    SRfmodels.append(est)

seed=42


/Users/swagatam/miniconda3/envs/hiwi/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression
Test accuracy: 0.8467 Test AUROC: 0.8193 Test F1 score: 0.3380
DecisionTreeClassifier
Test accuracy: 0.8700 Test AUROC: 0.8443 Test F1 score: 0.3926
RandomForestClassifier
Test accuracy: 0.9490 Test AUROC: 0.9315 Test F1 score: 0.2526
XGBClassifier
Test accuracy: 0.9346 Test AUROC: 0.9426 Test F1 score: 0.5806
Time to fit symbolic classifier: 67.33232116699219 seconds
SymbolicClassifier
Test accuracy: 0.9411 Test AUROC: 0.6109 Test F1 score: 0.3223
Time to fit symbolic classifier: 65.69597911834717 seconds
SymbolicClassifier
Test accuracy: 0.9447 Test AUROC: 0.5897 Test F1 score: 0.2870
seed=2024


/Users/swagatam/miniconda3/envs/hiwi/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression
Test accuracy: 0.8492 Test AUROC: 0.8519 Test F1 score: 0.3558
DecisionTreeClassifier
Test accuracy: 0.7745 Test AUROC: 0.8317 Test F1 score: 0.2991
RandomForestClassifier
Test accuracy: 0.9504 Test AUROC: 0.9401 Test F1 score: 0.3030
XGBClassifier
Test accuracy: 0.9318 Test AUROC: 0.9497 Test F1 score: 0.5662
Time to fit symbolic classifier: 66.04439783096313 seconds
SymbolicClassifier
Test accuracy: 0.9479 Test AUROC: 0.6489 Test F1 score: 0.4130
Time to fit symbolic classifier: 67.61290097236633 seconds
SymbolicClassifier
Test accuracy: 0.9483 Test AUROC: 0.6368 Test F1 score: 0.3950
seed=1234


...

LogisticRegression
Test accuracy: 0.8348 Test AUROC: 0.8110 Test F1 score: 0.3195
DecisionTreeClassifier
Test accuracy: 0.7907 Test AUROC: 0.8045 Test F1 score: 0.2899
RandomForestClassifier
Test accuracy: 0.9483 Test AUROC: 0.9151 Test F1 score: 0.2500
XGBClassifier
Test accuracy: 0.9293 Test AUROC: 0.9249 Test F1 score: 0.5343
Time to fit symbolic classifier: 75.61543703079224 seconds
SymbolicClassifier
Test accuracy: 0.9451 Test AUROC: 0.5476 Test F1 score: 0.1730
Time to fit symbolic classifier: 68.48590612411499 seconds
SymbolicClassifier
Test accuracy: 0.9461 Test AUROC: 0.5877 Test F1 score: 0.2857
seed=82273


/Users/swagatam/miniconda3/envs/hiwi/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression
Test accuracy: 0.8531 Test AUROC: 0.8229 Test F1 score: 0.3599
DecisionTreeClassifier
Test accuracy: 0.8456 Test AUROC: 0.8416 Test F1 score: 0.3364
RandomForestClassifier
Test accuracy: 0.9476 Test AUROC: 0.9164 Test F1 score: 0.2551
XGBClassifier
Test accuracy: 0.9268 Test AUROC: 0.9287 Test F1 score: 0.5256
Time to fit symbolic classifier: 68.64090394973755 seconds
SymbolicClassifier
Test accuracy: 0.9422 Test AUROC: 0.6260 Test F1 score: 0.3534
Time to fit symbolic classifier: 68.7185320854187 seconds
SymbolicClassifier
Test accuracy: 0.9436 Test AUROC: 0.6005 Test F1 score: 0.3084
seed=90378


/Users/swagatam/miniconda3/envs/hiwi/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression
Test accuracy: 0.8657 Test AUROC: 0.8305 Test F1 score: 0.3746
DecisionTreeClassifier
Test accuracy: 0.7899 Test AUROC: 0.7964 Test F1 score: 0.2875
RandomForestClassifier
Test accuracy: 0.9490 Test AUROC: 0.9247 Test F1 score: 0.2680
XGBClassifier
Test accuracy: 0.9311 Test AUROC: 0.9397 Test F1 score: 0.5450
Time to fit symbolic classifier: 68.76493072509766 seconds
SymbolicClassifier
Test accuracy: 0.9465 Test AUROC: 0.6394 Test F1 score: 0.3918
Time to fit symbolic classifier: 67.91902613639832 seconds
SymbolicClassifier
Test accuracy: 0.9487 Test AUROC: 0.6257 Test F1 score: 0.3755


In [37]:
LENGTHS = {'SR': [], 'SRf': []}
for est in SRmodels:
    LENGTHS['SR'].append(est._program.length_)
for est in SRfmodels:
    LENGTHS['SRf'].append(est._program.length_)
    
LENGTHS['SR'] = np.asarray(LENGTHS['SR'])
LENGTHS['SRf'] = np.asarray(LENGTHS['SRf'])

convert_to_arrays()

In [38]:
plusminus = pm = u"\u00B1"
for model_name in MODEL_NAMES:
    print(f"{model_name} acc: {ACCURACIES[model_name].mean():.2f} {pm} {ACCURACIES[model_name].std():.4f}")
print('--------')
for model_name in MODEL_NAMES:
    print(f"{model_name} F1: {F1SCORES[model_name].mean():.2f} {pm} {F1SCORES[model_name].std():.4f}")

print('--------')

LR acc: 0.85 ± 0.0087
DT acc: 0.80 ± 0.0397
RF acc: 0.95 ± 0.0017
XG acc: 0.93 ± 0.0045
SR acc: 0.95 ± 0.0022
SRf acc: 0.95 ± 0.0025
--------
LR F1: 0.35 ± 0.0202
DT F1: 0.31 ± 0.0395
RF F1: 0.29 ± 0.0393
XG F1: 0.56 ± 0.0248
SR F1: 0.35 ± 0.0708
SRf F1: 0.35 ± 0.0393
--------


In [39]:
# LENGTHS['SR'].mean(), LENGTHS['SRf'].mean()

In [40]:
print(f"SR mean length: {LENGTHS['SR'].mean()}")
print(f"SRf mean length: {LENGTHS['SRf'].mean()}")

SR mean length: 1.6
SRf mean length: 4.0


**Conclusion**: From the results above we see that symbolic regression models have F1 
score below 0.5, i.e., they have very poor classification performance. Hence for a 
fairer comparison among the classifiers, we do not use imbalanced data in our 
experiments.